# Playground

In [74]:
import os
import json
import copy

train_folder = 'data/training'
all_data = []

In [75]:
# Extracting Data from the Training Set
for filename in os.listdir(train_folder): 
    if filename.endswith('.json'):
        with open(os.path.join(train_folder, filename), 'r') as file:
            data = json.load(file)
            train_data = data['train']
            test_data = data['test']
            all_data.append([train_data, test_data])

print("Total Files:", len(os.listdir(train_folder)))
print("Total Data:", len(all_data))

Total Files: 400
Total Data: 400


In [93]:
def data_parser(user_data, hide_test_labels=False, return_answer=False):

    # Making a copy of data
    data = copy.deepcopy(user_data)
    train_samples = []
    test_sample = None
    train, test =  data

    # Parsing Samples
    for sample in train:
        train_samples.append(sample)
    for sample in test:
        answer = sample['output']
        if hide_test_labels:
            sample['output'] = '?'
            test_sample = sample
        else:
            test_sample = sample
            
    if return_answer:
        del data
        return train_samples, test_sample, answer
    else:
        del data
        return train_samples, test_sample

In [95]:
data_parser(all_data[5], hide_test_labels=False, return_answer=True)[2]

[[7]]